In [1]:
# Johannes Mellak investigating a different sign depending on whether using exact diag. or NetKet expectation values

import netket as nk
import numpy as np

# --------------------------------------------------------------------------------------------------------
# Prepare the basics (From NetKet/Examples/DissipativeIsing1d)

g = nk.graph.Hypercube(length=2, n_dim=1, pbc=False)

hi = nk.hilbert.Spin(s=1 / 2, N=g.n_nodes)
ha = nk.operator.LocalOperator(hi)
j_ops = []

# Observables
obs_sy = nk.operator.LocalOperator(hi, dtype=complex)

gp = 0.3
Vp = 2.0
for i in range(2):
    ha += (gp / 2.0) * nk.operator.spin.sigmax(hi, i)
    ha += (
        (Vp / 4.0)
        * nk.operator.spin.sigmaz(hi, i)
        * nk.operator.spin.sigmaz(hi, (i + 1) % 2)
    )
    # sigma_{-} dissipation on every site
    j_ops.append(nk.operator.spin.sigmam(hi, i))
    obs_sy += nk.operator.spin.sigmay(hi, i)

lind = nk.operator.LocalLiouvillian(ha, j_ops)

In [2]:
# --------------------------------------------------------------------------------------------------------
# Run the optimization

ma = nk.models.NDM(beta=1)
sa = nk.sampler.MetropolisLocal(hilbert=lind.hilbert)
optim = nk.optimizer.Sgd(learning_rate=0.04)

rho = nk.variational.MCMixedState(sampler=sa, model=ma, n_samples=500, n_samples_diag=500)
rho.init_parameters(nk.nn.initializers.normal(stddev=0.01))

ss = nk.SteadyState(lindbladian=lind, optimizer=optim, variational_state=rho)

ss.run(n_iter=300, out='results/testOp', obs={"Sy": obs_sy}) # converges in my simulations

# import visualizer        # check convergence
# vis = visualizer.Visualizer('results/testOp')
# vis.plot_observable("LdagL")
# vis.plot_observable("Sy")

# --------------------------------------------------------------------------------------------------------
# Compare the results for some observable

# The observable operator to investigate
obs = obs_sy

# Calculate exact result
rho_0 = nk.exact.steady_state(lind)

print('_'*80)
print(f'RBM result for the observable: {ss.estimate(obs)}')
print(f'Exact result from ED:          {np.trace(rho_0@obs.to_dense())}')

print('...these results have a different sign. (High uncertainty is due to the imaginary part)')
print('_'*80)

100%|██████████| 300/300 [00:06<00:00, 42.86it/s, LdagL=0.00142 ± 0.00045 [σ²=0.00009]]


Minimum eigenvalue is:  -3.7038045773014024e-16
________________________________________________________________________________
RBM result for the observable: 0.081-0.036j ± 0.063 [σ²=2.005, R̂=0.9999]
Exact result from ED:          (0.08141487086313604+9.298117831235686e-16j)
...these results have a different sign. (High uncertainty is due to the imaginary part)
________________________________________________________________________________


In [3]:
lind.to_dense().real

array([[-2. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ,
         0. ,  0. ,  0

In [4]:



# --------------------------------------------------------------------------------------------------------
# Digging deeper into the code of netket... (DEBUG CODE)
# 1.) I reproduce the code that calculates the expectation value and
# 2.) replace the RBM result of the density matrix rho with the exact rho_0
#
# => the expectation value gives the exact result with a negative sign for asymmetric observables (e.g. S_y)

def reproduceLocalValueCalculation(samples_u, operator, use_exact_rho=False):
    # Reproduce local value calculation from _local_cost_functions.py::198:local_value_op_op_cost()

    samples_uc, mels = operator.get_conn_padded(samples_u)

    import jax
    import jax.numpy as jnp

    locs = []
    for batch in range(samples_u.shape[0]):
        # Manually handling batches
        s = samples_u[batch]
        s_ind = hi.states_to_numbers(s)
        s_p = samples_uc[batch]
        p_ind = hi.states_to_numbers(s_p)
        mel = mels[batch]

        # from local_value_op_op_cost
        s_sp = jax.vmap(lambda samples, samples_p: jnp.hstack((samples, samples_p)), in_axes=(None, 0))(s, s_p)
        s_s = jnp.hstack((s, s))

        # Get the RBM values
        log_s_sp = rho._apply_fun(rho.variables, s_sp)
        log_s_s = rho._apply_fun(rho.variables, s_s)
        # Or better: Use the exact rho values from ED instead of the function values
        if use_exact_rho:
            log_s_sp = np.log(rho_0[s_ind][p_ind])
            log_s_s = np.log(rho_0[s_ind][s_ind])

        # Calculate local value
        locs.append(jnp.sum(mel * jnp.exp(log_s_sp - log_s_s)))

    return locs

In [5]:

# MCMC samples that were generated by the sampler
samples_r = np.asarray(rho.diagonal.samples.reshape((-1, rho.diagonal.samples.shape[-1])))

print('Calculating expectation value "manually" ... takes a minute')
locs_r = reproduceLocalValueCalculation(samples_r, obs)
print('_'*80)
print(f'Reproduction of expectation value: {np.mean(locs_r)}')
print(f'Netket expectation value:          {ss.estimate(obs).mean}')

print('check: the reproduction of the netket code works')

# to remove MC noise, uniform distribution of states
samples_uniform = hi.all_states()
locs_u = reproduceLocalValueCalculation(samples_uniform, obs, use_exact_rho=True)
# Correct the probability distribution of the uniform samples according to the diagonal of rho
result_u = np.sum(np.array(locs_u) * rho_0.diagonal())

print('_'*80)
print(f'Reproduction of expectation value using rho_0: {result_u}')
print(f'Result from exact diagonalization rho_0      : {np.trace(rho_0@obs.to_dense())}')
print('...again, these results have a different sign')

print('_'*80)
print('finished')

Calculating expectation value "manually" ... takes a minute
________________________________________________________________________________
Reproduction of expectation value: (-0.08054180566665735-0.035762756557586484j)
Netket expectation value:          (0.08054180566665728-0.0357627565575864j)
check: the reproduction of the netket code works
________________________________________________________________________________
Reproduction of expectation value using rho_0: (-0.08141487086313602-8.743006318923108e-16j)
Result from exact diagonalization rho_0      : (0.08141487086313604+9.298117831235686e-16j)
...again, these results have a different sign
________________________________________________________________________________
finished
